In [1]:
import os
import json
import uuid
from transformers import AutoTokenizer, AutoModel
from milvus import default_server
from pymilvus import (
    connections, utility, Collection,
    CollectionSchema, FieldSchema, DataType
)
import torch

In [2]:
class VectorSearchWrapper:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        if torch.cuda.is_available():
            self.device = torch.device("cuda:0")  # Use GPU if CUDA is available

        else:
            self.device = torch.device("cpu")
        self.EMBED_MODEL = 'sentence-transformers/all-mpnet-base-v2'
        self.DIM = 768
        self.json_file_path='F:\\Chat_Proj\\Project Files\\combining and automation\\cleaned_and_combined_hyd.json'
        self.collection = None
        self.collection_name = "chat_demo"
        self.MILVUS_URI = 'http://localhost:19530'
        [self.MILVUS_HOST, self.MILVUS_PORT] = self.MILVUS_URI.split('://')[1].split(':')
        self.result=""
        self.embeddings=dict()
        self.history=list()
        self.run()
    def initialize_model(self):
        # Load the pre-trained model and tokenizer
        model_name = self.EMBED_MODEL
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
    def embedding(self,text_data):

        inputs = self.tokenizer(text_data, return_tensors='pt', padding=True, truncation=True)
        if self.device.type == 'cuda':
            inputs = {key: tensor.cuda() for key, tensor in inputs.items()}  # Move tensors to CUDA
        if self.device.type == 'xla':
            inputs = {key: tensor.to(self.device) for key, tensor in inputs.items()}# Move tensors to TPU
        with torch.no_grad():
            # Forward pass through the model
            outputs = self.model(**inputs)


        embeddings = outputs.last_hidden_state.mean(dim=1)  # Assuming you want to use mean pooling

        # Normalize the embeddings if needed
        embeddings=embeddings[0]

        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=-1)

        return normalized_embeddings
    
    def preprocess_and_embed(self):
        embedded_list = {}
        with open(self.json_file_path, 'r') as file:
            json_data = json.load(file)

        # Preprocess and embed each JSON entry
        for entry in json_data:
            curr_dict = {}
            text_data = entry["text data"]
            entry_id = entry.get("Id", None)  # Extract the ID from JSON (assuming it has an "id" field)
            
            if text_data not in embedded_list:  # Avoid duplicate embeddings
                embedding = self.embedding(text_data)
                embedded_list[entry_id] = {
                    "embedding": embedding,
                    "text": text_data
                }

        return embedded_list
    def create_collection(self,collection_name):
        connections.connect(host=self.MILVUS_HOST, port=self.MILVUS_PORT)

        has_collection = utility.has_collection(collection_name)

        if has_collection:
            utility.drop_collection(collection_name)
        # Create collection
        fields = [
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True,auto_id=False),
            FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=self.DIM),
            FieldSchema(name='text', dtype=DataType.VARCHAR, max_length=1000)
        ]
        schema = CollectionSchema(
            fields=fields,
            description="Towhee demo",
            enable_dynamic_field=True
        )
        collection = Collection(name=collection_name, schema=schema)

        # Change index here if you want to accelerate search
        index_params = {
            'metric_type': 'IP',
            'index_type': 'IVF_FLAT',
            'params': {'nlist': 4096,'nprobe':256}
        }
        collection.create_index(
            field_name='embedding',
            index_params=index_params
        )

        return collection

    def prepare_data(self):
        data = []
        embedded_data = self.embeddings

        for entry_id, entry in embedded_data.items():
            entity = {
                'id': entry_id,  # Use the provided ID
                'embedding': entry['embedding'],
                'text': entry['text']
            }
            data.append(entity)

        return data
    def insert_data(self,collection_name):
        self.collection = self.create_collection(collection_name)

        # Prepare data
        data_to_insert = self.prepare_data()
        # Insert data into the collection
        self.collection.insert(data_to_insert)
    def run(self,collection_name=None):
        if collection_name is None:
            collection_name = self.collection_name

        # Initialize model asynchronously
        self.initialize_model()
        

        # Preprocess and embed data asynchronously
        self.embeddings =self.preprocess_and_embed()

        # Insert data asynchronously
        self.insert_data(collection_name)
        
        
    def search_milvus(self,query):
        embedded_vec=self.embedding(query).cpu().numpy()
        collection=Collection(name=self.collection_name)
        collection.load()
        res=collection.search(
            data=[embedded_vec],
            anns_field="embedding",
            param={
            'metric_type': 'IP',
            'params': {'nlist': 4096}
                    },
            limit=3,

            output_fields=["text"]   )

        text_li=list()
        id_li=list()
        dist_li=list()
        for i, hits in enumerate(res):

            for hit in hits:
                id_li.append(hit.entity.id)
                dist_li.append(hit.entity.distance)
                text_li.append(hit.entity.get("text"))
        data=dict()
        data["id"]=id_li
        data["dist"]=dist_li
        data["text"]=text_li
        return data


In [4]:
default_server.start()

In [5]:
chatbot=VectorSearchWrapper()

In [7]:
data=chatbot.search_milvus(query="CBSE schools  Location:dilsukhnagar")

In [8]:
print(data)

{'id': [880, 1882, 1592], 'dist': [0.8021835088729858, 0.7947474718093872, 0.7850978374481201], 'text': ['Name: DILSUKHNAGAR PUBLIC SCHOOL~Category: Other~Location: BN Reddy Colony, Vanasthalipuram, 5-6-525, Rd Number 9/5, Sri Krishna Devaraya Nagar, B.N Reddy Nagar, Hyderabad, Telangana 500070, India~Faculty: ~Sports: ~Amenities: Transport~Board: CBSE~Years: 0~Fee: 34000~Since: Not Available~Strength: Not Available~', 'Name: DILSUKHNAGAR PUBLIC SCHOOL~Category: Other~Location: 8GVH+6XH, Dps school line, Sri Raghavendra Nagar Colony, Jyothi Nagar, Kharmanghat, Telangana 500079, India~Faculty: ~Sports: ~Amenities: Transport~Board: CBSE~Years: 0~Fee: 40000~Since: Not Available~Strength: Not Available~', 'Name: DILSUKHNAGAR PUBLIC SCHOOL~Category: Other~Location: Alkapuri-RK Puram Road, polkampally, Ramakrishnapuram, L. B. Nagar, Hyderabad, Telangana 500035, India~Faculty: ~Sports: ~Amenities: Transport~Board: State Board~Years: 0~Fee: 34000~Since: Not Available~Strength: Not Available~']

In [9]:
default_server.stop()
